<a href="https://colab.research.google.com/github/gogifemi/cannabis-health-diagnosis/blob/main/Running_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/gogifemi/cannabis-health-diagnosis.git

Cloning into 'cannabis-health-diagnosis'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 4), reused 17 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 2.14 MiB | 20.28 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [4]:
!pip install tensorflow

In [6]:
!unzip /content/cannabis-health-diagnosis/data.zip -d /content/

Archive:  /content/cannabis-health-diagnosis/data.zip
   creating: /content/test/
   creating: /content/test/Aphids/
  inflating: /content/test/Aphids/cannabis-marijuana-aphids-outdoor-garden-pests_RQ_jpeg.rf.1d58d5f5e1d120fd5ecad31e9f22a176.jpg  
  inflating: /content/test/Aphids/OIP_jpeg.rf.2d6b866972b2d670a04c1d7b0c3910aa.jpg  
   creating: /content/test/Calcium Deficiency/
  inflating: /content/test/Calcium Deficiency/cannabis-calcium-deficiency-nutrient-deficiencies-yellow-brown-spots-calmag-weed-growing-problems_3Z_jpeg.rf.69898993d8c98cfbe550cc1e0926efd5.jpg  
  inflating: /content/test/Calcium Deficiency/cannabis-calcium-deficiency-nutrient-deficiencies-yellow-brown-spots-calmag-weed-growing-problems_DX_png.rf.cfc31d70ebfb8fe75654bb86966d40f5.jpg  
  inflating: /content/test/Calcium Deficiency/cannabis-calcium-deficiency-nutrient-deficiencies-yellow-brown-spots-calmag-weed-growing-problems_HI_jpeg.rf.14170eedbfa5e3bc79ae8c86dfa68f02.jpg  
  inflating: /content/test/Calcium Defi

In [9]:
!ls /content/cannabis-health-diagnosis/data/

test  train  valid


In [10]:
data_dir = '/content/cannabis-health-diagnosis/data/'

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Training**

In [13]:
from tensorflow.keras import Sequential,Input,Model
from tensorflow.keras.layers import Dense,Dropout,GlobalAveragePooling2D,Flatten
from tensorflow.keras.preprocessing import image # Keras own inbuild image class
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
from tensorflow.keras.metrics import Precision, Recall

In [11]:
from tensorflow.keras.optimizers import Adam

In [35]:
from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow import keras as K
import tensorflow.keras.backend as K
K.clear_session()

#1. Select the convolutional base / Pretrained network
base_model = VGG16(include_top=False, input_shape=(416, 416, 3))

In [15]:
base_model.trainable= False

In [17]:
base_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None, None, 3)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, None, None, 64)      │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, None, None, 64)      │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, None, None, 64)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, None, None, 128)     │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, None, None, 128)     │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, None, None, 128)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, None, None, 256)     │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, None, None, 256)     │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, None, None, 256)     │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, None, None, 256)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, None, None, 512)     │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, None, None, 512)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, None, None, 512)     │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [18]:
inputs = Input(shape=(416,416,3))

base = base_model(inputs)

flatten = Flatten()(base)

outputs = Dense(12,activation='softmax')(flatten)

model_tf = Model(inputs,outputs)

In [19]:
model_tf.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 416, 416, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg16 (Functional)                   │ (None, 13, 13, 512)         │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 12)                  │       1,038,348 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,753,036 (60.09 MB)

 Trainable params: 1,038,348 (3.96 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [20]:
classes = [
    'Healthy',
    'Manganese Deficiency',
    'Phosphorus Deficiency',
    'Potassium Deficiency',
    'Calcium Deficiency',
    'PH Fluctuation',
    'Sulfur Deficiency',
    'Iron Deficiency',
    'Nitrogen Deficiency',
    'Aphids',
    'Magnesium Deficiency',
    'Powdery Mildew'
]

In [21]:
data_gen = image.ImageDataGenerator(
    preprocessing_function=preprocess_input,
    fill_mode='nearest',
    rotation_range=360,
)

In [46]:
train_data_gen = data_gen.flow_from_directory(
    directory='/content/cannabis-health-diagnosis/data/train',
    class_mode="categorical",
    classes=classes,
    batch_size=5,
    target_size=(416, 416),
)

Found 955 images belonging to 12 classes.


In [47]:
val_data_gen = data_gen.flow_from_directory(
    directory='/content/cannabis-health-diagnosis/data/valid',
    class_mode="categorical",
    classes=classes,
    batch_size=5,
    target_size=(416, 416),
)

Found 35 images belonging to 12 classes.


In [24]:
from tensorflow.keras.callbacks import EarlyStopping

In [33]:
!pip install --upgrade tensorflow

In [37]:
!pip show keras

Name: keras
Version: 3.4.1
Summary: Multi-backend Keras.
Home-page: https://github.com/keras-team/keras
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, packaging, rich
Required-by: tensorflow


In [38]:
!pip show tensorflow

Name: tensorflow
Version: 2.17.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras


In [39]:
import keras

In [ ]:
model_tf.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model_tf.fit(train_data_gen,
             verbose=2,
             epochs=20,
             validation_data=val_data_gen,
             callbacks=[early_stopping])